In [1]:
from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [178]:
import ollama

response = ollama.generate(
    model="codeqwen:7b-custom",
    prompt="What is 8*8"
)
print(response["response"])


The product of 8 and 8 is 64.


In [ ]:
test_df = pd.read_csv('../Splits/combined/sub_test.csv')
# test_df = pd.read_csv('../Splits/harder_test/sub_test.csv')
test_df

,Unnamed: 0.1,writer,problem_id,submission_id,website,source,Unnamed: 0
0,0,AI,164A,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN
1,1,AI,NaN,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN
2,2,AI,o61_may08_estate,NaN,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,1843.0
3,3,human,NaN,13646458.0,https://codeforces.com,//Template\n\n// By Anudeep :)\n//Includes\n#i...,NaN
4,4,AI,1032,NaN,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,998.0
...,...,...,...,...,...,...,...
2995,2995,human,1000,306681.0,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,NaN
2996,2996,human,1055,310229.0,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...,NaN
2997,2997,Human,NaN,NaN,https://firefly.gchan.moe,#include <bits/stdc++.h>\n\nusing namespace st...,NaN
2998,2998,AI,888F,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,NaN


In [7]:
# Prompt URL
prompt_path = "./Prompts/PromptV1.txt"

with open(prompt_path,'r') as _prompt:
  prompt = _prompt.read()
prompt

'You are the best C++ code detector. Your job is to determine whether the competitive programming source code is AI-generated or written by a human.\n\n### Steps:\n1. **Read the source code carefully** to understand what does the code doing.\n2. **Identify the patterns of code** and analyze whter it\'s AI-generated or not, like checking unnesscary space, variables, new lines, or comments.\n3. **Determine if it\'s Human or AI** from the data you have.\n4. **From the reason** Answer if it AI or Human.\n\nYou must respond in JSON format, like this:\n```json\n{   \n    "reason": "Why this code is human-written or AI-generated"\n    "answer": "human" / "AI",  \n}\n```\n- You must not provide any other text, except JSON.\n- In the reason part, don\'t use symbol like " , ` , \\n, use only normal character.\nHere is C++ source code:\n'

In [181]:
len(prompt_path + test_df['source'][4])

1856

In [182]:
response = ollama.chat(
    model='codeqwen:7b-custom',
    messages=[
        {'role': 'system', 'content': prompt},
        {'role': 'user', 'content': test_df['source'][5]},
        ]
    )

In [183]:
response.message.content

'{"reason": "The code is a C++ program that plays the game Othello (Reversi). It uses functions to place pieces, check if a move can be made by a certain color, and determine if the board is full. The code contains nested loops for checking all possible moves for each color.","answer": "human"}'

In [184]:
def GenerateResult(source: str):
  try:
    response = ollama.chat(
    model='codeqwen:7b-custom',
    messages=[
        {'role': 'system', 'content': prompt},
        {'role': 'user', 'content': source},
        ]
    )
    return response.message.content.lstrip('```json\n').rstrip('\n```')
  except Exception as e:
    print(f"ERROR : {e}")
    return None

In [2]:
BACKUP_PATH = './Backups/Result-CodeQwen7b.csv'

def save_responses(res):
    df = pd.DataFrame(res, columns=['back_up'])
    df.to_csv(BACKUP_PATH,index=False)

In [3]:
try:
    saved_df = pd.read_csv(BACKUP_PATH)
    answer = saved_df['back_up'].to_list()
except:
    answer = []

In [4]:
saved_df

,back_up
0,"{""reason"": ""This code uses the Breadth-First S..."
1,"{""reason"": ""This code seems to be a modified v..."
2,"{""reason"": ""The code defines a function dfs th..."
3,"{""reason"": ""The code has a structure that invo..."
4,"{""reason"": ""The code is a competitive programm..."
...,...
2995,"{""reason"": ""The code uses short variables and ..."
2996,"{""reason"": ""This code seems to be an implement..."
2997,"{""reason"": ""This code provides a helper functi..."
2998,"{""reason"": ""This code is AI generated as it co..."


In [10]:
len(answer)

2950

In [188]:
import pathlib
import httpx
import time
from IPython.display import clear_output
err_logs = []
answer = []

err_cnt = 0

for index, row in test_df[len(answer):].iterrows():
  while True:
    if index%50 == 0 and index != 0:
      save_responses(answer)
    if err_cnt >= 3:
      answer.append('{"reason": "ERROR","answer": "ERROR"}')
      err_cnt =0
      break
    try:
      response = GenerateResult(row['source'])
      clear_output()
      print(f"ASKED : {index}")
      print(json.loads(response))
      err_cnt = 0
      answer.append(response)
      break
    except Exception as e:
      err_cnt += 1
      err_logs.append((index,e))
      clear_output()
      save_responses(answer)
      print(response)
      print(f"ERROR at {index} : {e}")

ASKED : 2999
{'reason': "The code contains loops and conditional statements to generate all possible triplets of consecutive grid cells. It checks for three consecutive 'x' characters in each direction (horizontal, vertical, main diagonal, and anti-diagonal) to determine if a player can win by placing a single 'x'. The variables used are short and simple, with meaningful names indicating their purpose.", 'answer': 'human'}


In [23]:
import re
import json


result_n = len(answer)
answer_df = test_df.iloc[:result_n]
answer_df['answer_text'] = answer

C:\Users\Pakin\AppData\Local\Temp\ipykernel_26416\3307430667.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_text'] = answer


In [24]:
answer_df['answer_dict'] = answer_df['answer_text'].apply(json.loads)
answer_df['model_answer'] = answer_df['answer_dict'].apply(lambda x: x['answer'])
answer_df['model_reason'] = answer_df['answer_dict'].apply(lambda x: x['reason'])
answer_df['verdict'] = answer_df['writer'] == answer_df['model_answer']
answer_df = answer_df[['writer','model_answer','verdict','model_reason','website','source']]

C:\Users\Pakin\AppData\Local\Temp\ipykernel_26416\3639045823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_dict'] = answer_df['answer_text'].apply(json.loads)
C:\Users\Pakin\AppData\Local\Temp\ipykernel_26416\3639045823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['model_answer'] = answer_df['answer_dict'].apply(lambda x: x['answer'])
C:\Users\Pakin\AppData\Local\Temp\ipykernel_26416\3639045823.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [11]:
answer_df

,writer,model_answer,verdict,model_reason,website,source
0,AI,human,False,This code uses the Breadth-First Search algori...,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...
1,AI,human,False,This code seems to be a modified version of th...,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...
2,AI,human,False,The code defines a function dfs that performs ...,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...
3,human,human,True,The code has a structure that involves functio...,https://codeforces.com,//Template\n\n// By Anudeep :)\n//Includes\n#i...
4,AI,human,False,The code is a competitive programming problem-...,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...
...,...,...,...,...,...,...
2995,human,human,True,The code uses short variables and basic string...,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...
2996,human,human,True,This code seems to be an implementation of a s...,https://programming.in.th,#include <bits/stdc++.h>\nusing namespace std;...
2997,Human,human,False,This code provides a helper function to check ...,https://firefly.gchan.moe,#include <bits/stdc++.h>\n\nusing namespace st...
2998,AI,AI,True,This code is AI generated as it contains loops...,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...


In [41]:
filtered_df = pd.DataFrame()
filtered_df[['Label','Model_Answer','Verdict','Reason','Website']] = answer_df[['writer','model_answer','verdict','model_reason','website']]

In [43]:
filtered_df['Model_Answer'] = filtered_df['Model_Answer'].replace('Human', 'human')
filtered_df['Label'] = filtered_df['Label'].replace('Human', 'human')
filtered_df['Verdict'] = filtered_df['Label']==filtered_df['Model_Answer']
filtered_df = filtered_df[filtered_df['Model_Answer'].isin(['human', 'AI'])]
filtered_df

,Label,Model_Answer,Verdict,Reason,Website
0,AI,human,False,This code uses the Breadth-First Search algori...,https://codeforces.com
1,AI,human,False,This code seems to be a modified version of th...,https://codeforces.com
2,AI,human,False,The code defines a function dfs that performs ...,https://programming.in.th
3,human,human,True,The code has a structure that involves functio...,https://codeforces.com
4,AI,human,False,The code is a competitive programming problem-...,https://programming.in.th
...,...,...,...,...,...
2995,human,human,True,The code uses short variables and basic string...,https://programming.in.th
2996,human,human,True,This code seems to be an implementation of a s...,https://programming.in.th
2997,human,human,True,This code provides a helper function to check ...,https://firefly.gchan.moe
2998,AI,AI,True,This code is AI generated as it contains loops...,https://codeforces.com


In [44]:
filtered_df['Model_Answer'].value_counts()

Model_Answer
human    2717
AI        279
Name: count, dtype: int64

In [45]:
filtered_df.to_csv("./Results/_2/Result-CodeQwen7b.csv",index=False)

In [40]:
accuracy = filtered_df['Verdict'].value_counts(normalize=True).get(True, 0)
print(f"AUC : {accuracy}")

AUC : 0.4836666666666667
